In [1]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import webbrowser
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pytesseract
from PIL import Image

# Initialize the recognizer and speech engine
listener = sr.Recognizer()
machine = pyttsx3.init()

# Function to speak text
def talk(text):
    machine.say(text)
    machine.runAndWait()

# Function to listen for user input
def input_instruction():
    instruction = ""
    try:
        with sr.Microphone() as source:
            print("Listening...")
            audio = listener.listen(source)
            instruction = listener.recognize_google(audio)
            instruction = instruction.lower()
            if "Jason" in instruction:
                instruction = instruction.replace('Jason', '')
                print(instruction)
    except sr.UnknownValueError:
        pass
    return instruction

# Function to get weather information
def get_weather(city, state):
    try:
        # Replace with the National Weather Service API endpoint for your location
        api_url = f"https://api.weather.gov/points/karnataka/bengaluru"
        response = requests.get(api_url)
        data = response.json()

        forecast_url = data['properties']['forecast']
        forecast_response = requests.get(forecast_url)
        forecast_data = forecast_response.json()
        
        # Extract the current weather conditions
        current_conditions = forecast_data['properties']['periods'][0]['detailedForecast']
        return current_conditions
    except Exception as e:
        print(e)
        return "Sorry, I couldn't fetch the weather information."

# Function to send an email
def send_email(subject, body, recipient_email):
    try:
        # Email configuration
        smtp_server = 'smtp.example.com'  # Replace with your SMTP server address
        smtp_port = 587  # Replace with your SMTP server port (587 for TLS, 465 for SSL)
        sender_email = 'your_email@example.com'  # Your email address
        sender_password = 'your_password'  # Your email password

        # Create a MIME object to represent the email
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject

        # Email body
        message.attach(MIMEText(body, 'plain'))

        # Establish an SMTP connection
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Use TLS encryption
        server.login(sender_email, sender_password)
        
        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())
        
        server.quit()
        
        talk('Email sent successfully!')
    except Exception as e:
        print('An error occurred:', str(e))
        talk('Sorry, I couldn\'t send the email.')

# Function to create a calendar event
def create_event(event_date, event_summary):
    try:
        # Implement your calendar integration here (e.g., Google Calendar API)
        # Add code to create an event on the specified date with the given summary
        # Example code: ...
        talk(f"Event '{event_summary}' has been added to your calendar on {event_date}.")
    except Exception as e:
        print(e)
        talk("Sorry, I couldn't add the event to your calendar.")

# Function to control system actions
def system_control(instruction):
    if 'open notepad' in instruction:
        talk("Opening Notepad")
        # Implement code to open Notepad or any other application as needed
    elif 'shutdown' in instruction:
        talk("Shutting down the computer")
        # Implement code to shut down the computer
    # Add more system control commands as needed

# Function to perform OCR on an image
def ocr_image(image_path):
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(e)
        return "Sorry, I couldn't perform OCR on the image."

# Main function
def play_Jason():
    instruction = input_instruction()
    print(instruction)

    if "play" in instruction:
        song = instruction.replace("play", "")
        talk("Playing " + song)
        pywhatkit.playonyt(song)

    elif 'time' in instruction:
        current_time = datetime.datetime.now().strftime('%I:%M %p')
        talk('Current time is ' + current_time)

    elif 'date' in instruction:
        current_date = datetime.datetime.now().strftime('%d/%m/%Y')
        talk("Today's date is " + current_date)

    elif 'how are you' in instruction:
        talk('I am fine, how about you?')

    elif 'what is your name' in instruction:
        talk('I am Jason, what can I do for you?')

    elif 'who is' in instruction:
        human = instruction.replace('who is', '')
        info = wikipedia.summary(human, 1)
        print(info)
        talk(info)

    elif 'weather' in instruction:
        talk("Please tell me the city and state for the weather information.")
        city_state = input_instruction().split()
        if len(city_state) == 2:
            city, state = city_state
            weather_info = get_weather(city, state)
            talk(f"The weather in {city}, {state} is: {weather_info}")
        else:
            talk("I couldn't understand the city and state for the weather.")
    
    elif 'send email' in instruction:
        talk('What is the subject of the email?')
        email_subject = input_instruction()
        talk('What should be the content of the email?')
        email_body = input_instruction()
        talk('To whom should I send the email?')
        recipient_email = input_instruction()
        
        send_email(email_subject, email_body, recipient_email)

    elif 'map' in instruction:
        location = instruction.replace('map', '').strip()
        talk(f"Opening Google Maps for {location}")
        webbrowser.open(f"https://www.google.com/maps/place/{location}")
        
    elif 'help' in instruction:
        help_message = """
        You can use the following commands:
        - Play [song name]
        - What is the time?
        - What is today's date?
        - How are you?
        - What is your name?
        - Who is [person]?
        - Weather
        - Map [location]
        - Send email
        - Help (to see this list)
        - Read text from image [image path]
        - Create a calendar event on [date] with summary [event summary]
        - Shutdown the computer
        - Open Notepad
        """
        talk(help_message)

    elif 'exit' in instruction:
        talk("Goodbye!")

    elif 'read text from image' in instruction:
        talk("Please provide the path to the image you want to perform OCR on.")
        image_path = input_instruction()
        ocr_text = ocr_image(image_path)
        talk("Here is the text from the image:")
        talk(ocr_text)

    elif 'create a calendar event' in instruction:
        talk("Please specify the date for the event (e.g., 'tomorrow at 3 PM'):")
        event_date = input_instruction()
        talk("Please provide a brief summary for the event:")
        event_summary = input_instruction()
        create_event(event_date, event_summary)
    
    elif 'system control' in instruction:
        system_control(instruction)

    else:
        talk('Can you please repeat yourself')

# Start the assistant
play_Jason()


Listening...



In [2]:
pip install pytesseract